In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from pyvi import ViTokenizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import str
import string

('liu', '888', 'bin', 'liu', '888', 'bin')
{'one': 1, 'two': 2}
dict_values([1, 2])


### Data

In [100]:
class Data(object):
    
    def load_raw_data(self, filename, is_train=True):
        ID = []
        label = []
        sentence = []
        review = ''        
        if is_train:
            with open(filename, 'r', encoding='utf-8') as file:
                for line in file:
                    if line.startswith('train_'):
                        ID.append(line.replace('\n',''))
                    elif line[:-1].isdigit():
                        review = review.replace('\n',' ')
                        review = ' '.join((review.lower()).split())
                        sentence.append(review)
                        review = ''
                        label.append(int(line[:-1]))
                    else:
                        review = review +' '+ line
        else:
            with open(filename, 'r', encoding='utf-8') as file:
                for line in file:
                    if line.startswith('test_'):
                        review = review.replace('\n',' ')
                        review = ' '.join((review.lower()).split())
                        sentence.append(review)
                        review = ''
                        ID.append(line.replace('\n',''))
                    else:
                        review = review +' '+ line
            review = review.replace('\n',' ')
            review = re.sub('['+string.punctuation+']', ' ', review)
            review = ' '.join((review.lower()).split())
            sentence.append(review)
            del sentence[0]
        return (ID, label, sentence)
    
    def pre_processing(self, text):
        
        # Xoa icon linh tinh
        emoji_pattern = re.compile("["
                       u"\U0001F600-\U0001F64F"  # emoticons
                       u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                       u"\U0001F680-\U0001F6FF"  # transport & map symbols
                       u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                       u"\U00002702-\U000027B0"
                       u"\U000024C2-\U0001F251"
                       u"\U0001f926-\U0001f937"
                       u"\u200d"
                       u"\u2640-\u2642" 
                       "]+", flags=re.UNICODE)
        text = emoji_pattern.sub(r'', text)
        
        # Chuan hoa tieng viet, tieng anh, thuat ngu
        replace_list={
            'òa': 'oà', 'óa': 'oá', 'ỏa': 'oả', 'õa': 'oã', 'ọa': 'oạ', 'òe': 'oè', 'óe': 'oé','ỏe': 'oẻ',
        'õe': 'oẽ', 'ọe': 'oẹ', 'ùy': 'uỳ', 'úy': 'uý', 'ủy': 'uỷ', 'ũy': 'uỹ','ụy': 'uỵ', 'uả': 'ủa',
        'ả': 'ả', 'ố': 'ố', 'u´': 'ố','ỗ': 'ỗ', 'ồ': 'ồ', 'ổ': 'ổ', 'ấ': 'ấ', 'ẫ': 'ẫ', 'ẩ': 'ẩ',
        'ầ': 'ầ', 'ỏ': 'ỏ', 'ề': 'ề','ễ': 'ễ', 'ắ': 'ắ', 'ủ': 'ủ', 'ế': 'ế', 'ở': 'ở', 'ỉ': 'ỉ',
        'ẻ': 'ẻ', 'àk': u' à ','aˋ': 'à', 'iˋ': 'ì', 'ă´': 'ắ','ử': 'ử', 'e˜': 'ẽ', 'y˜': 'ỹ', 'a´': 'á',
        #Chuẩn hóa 1 số sentiment words/English words
        ':))': '  positive ', ':)': ' positive ', 'ô kêi': ' ok ', 'okie': ' ok ', ' o kê ': ' ok ',
        'okey': ' ok ', 'ôkê': ' ok ', 'oki': ' ok ', ' oke ':  ' ok ',' okay':' ok ','okê':' ok ',
        ' tks ': u' cám ơn ', 'thks': u' cám ơn ', 'thanks': u' cám ơn ', 'ths': u' cám ơn ', 'thank': u' cám ơn ', 'cam on':u'cám ơn',u'cảm ơn':'cám ơn',
        'kg ': u' không ','not': u' không ', u' kg ': u' không ', '"k ': u' không ',' kh ':u' không ','kô':u' không ','hok':u' không ',' kp ': u' không phải ',u' kô ': u' không ', '"ko ': u' không ', u' ko ': u' không ', u' k ': u' không ', 'khong': u' không ', u' hok ': u' không ','k ':u' không ',
        'he he': ' positive ','hehe': ' positive ','hihi': ' positive ', 'haha': ' positive ', 'hjhj': ' positive ',
        ' lol ': ' negative ',' cc ': ' negative ','cute': u' dễ thương ','huhu': ' negative ', ' vs ': u' với ', 'wa': ' quá ', 'wá': u' quá', 'j': u' gì ', '“': ' ',
        ' sz ': u' cỡ ', 'size': u' cỡ ', u' đx ': u' được ', 'dk': u' được ', 'dc': u' được ', 'đk': u' được ',
        'đc': u' được ','authentic': u' chuẩn chính hãng ',u' aut ': u' chuẩn chính hãng ', u' auth ': u' chuẩn chính hãng ', 'thick': u' positive ', 'store': u' cửa hàng ',
        'shop': u' cửa hàng ', 'sp': u' sản phẩm ', 'gud': u' tốt ','god': u' tốt ','wel done':' tốt ', 'good': u' tốt ', 'gút': u' tốt ',
        'sấu': u' xấu ','gut': u' tốt ', u' tot ': u' tốt ', u' nice ': u' tốt ', 'perfect': 'rất tốt', 'bt': u' bình thường ',
        'time': u' thời gian ', 'qá': u' quá ', u' ship ': u' giao hàng ', u' m ': u' mình ', u' mik ': u' mình ',
        'ể': 'ể', 'product': 'sản phẩm', 'quality': 'chất lượng','chat':' chất ', 'excelent': 'hoàn hảo', 'bad': 'tệ','fresh': ' tươi ','sad': ' tệ ',
        'date': u' hạn sử dụng ', 'hsd': u' hạn sử dụng ','quickly': u' nhanh ', 'quick': u' nhanh ','fast': u' nhanh ','delivery': u' giao hàng ',u' síp ': u' giao hàng ',
        'beautiful': u' đẹp tuyệt vời ', u' tl ': u' trả lời ', u' r ': u' rồi ', u' shopE ': u' cửa hàng ',u' order ': u' đặt hàng ',
        'chất lg': u' chất lượng ',u' sd ': u' sử dụng ',u' dt ': u' điện thoại ',u' nt ': u' nhắn tin ',u' tl ': u' trả lời ',u' sài ': u' xài ',u'bjo':u' bao giờ ',
        'thik': u' thích ',u' sop ': u' cửa hàng ', ' fb ': ' facebook ', ' face ': ' facebook ', ' very ': u' rất ',u'quả ng ':u' quảng  ',
        'dep': u' đẹp ',u' xau ': u' xấu ','delicious': u' ngon ', u'hàg': u' hàng ', u'qủa': u' quả ', 
        'iu': u' yêu ','fake': u' giả mạo ', 'trl': 'trả lời', '><': u' positive ', 
        ' por ': u' tệ ',' poor ': u' tệ ', 'ib':u' nhắn tin ', 'rep':u' trả lời ',u'fback':' feedback ','fedback':' feedback ',
        'qc':u'quảng cáo',u'éo':'negative',
        #dưới 3* quy về 1*, trên 3* quy về 5*
        '6 sao': ' 5star ','6 star': ' 5star ', '5star': ' 5star ','5 sao': ' 5star ','5sao': ' 5star ',
        'starstarstarstarstar': ' 5star ', '1 sao': ' 1star ', '1sao': ' 1star ','2 sao':' 1star ','2sao':' 1star ',
        '2 starstar':' 1star ','1star': ' 1star ', '0 sao': ' 1star ', '0star': ' 1star ','1*':' 1star ', '2*':' 1star ','3*':' 5star ','4*':' 5star ','5*':' 5star '
        }
        for k, v in replace_list.items():
            text = text.replace(k, v)
        
        #Xóa dấu câu linh tinh
        text = re.sub('['+string.punctuation+']', ' ', text)
        
        # Tokenizer
        text = ViTokenizer.tokenize(text)
        
        return text
    
    
    def load_data(self, filename, is_train=True):
        ID, Label, Sentence = self.load_raw_data(filename, is_train)
        for i in range(len(Sentence)):
            Sentence[i] = self.pre_processing(Sentence[i])
        dic = {'0':['5star', 'positive'], '1':['1star', 'negative']}
        for key, vals in dic.items():
            for val in vals:
                ID.append((len(ID)+1))
                Label.append(key)
                Sentence.append(val)
        return ID, Label, Sentence

### Multinomial Naive Bayes Model

In [101]:
from sklearn.naive_bayes import MultinomialNB

In [102]:
class NaiveBayesModel(object):
    def __init__(self):
        self.clf = self._init_pipeline()
    
    @staticmethod
    def _init_pipeline():
        stopwords=('rằng', 'thì', 'mà', 'là', 'thế', 'à', 'ừ', 'vậy', 'như')
        pipe_line = Pipeline([
            ("vectorizer", CountVectorizer(stop_words=stopwords)), # bag of words
            ("tfidf", TfidfTransformer()), # tf-idf
            ("clf", MultinomialNB()) # navie bayes models
        ])
        return pipe_line

### Support Vector Machine - kernel = 'linear'

In [103]:
from sklearn.svm import SVC 

In [104]:
class SVMModel(object):
    def __init__(self):
        self.clf = self._init_pipeline()
        
    @staticmethod
    def _init_pipeline():
        stopwords=('rằng', 'thì', 'mà', 'là', 'thế', 'à', 'ừ', 'vậy', 'như')
        pipe_line = Pipeline([
            ("vectorizer", CountVectorizer(stop_words=stopwords)), # bag of words
            ("tfidf", TfidfTransformer()), # tf-idf
            ("clf_svm", SVC(kernel='linear')) # svm kernel = 'linear'
        ])
        return pipe_line

## Classifier

### Create data frame

In [105]:
ID, Label, Sentence = Data().load_data(r'D:\Documents\Đồ án 2\Sentiment Analysis\Sentiment data\vibo\train.crash')

In [106]:
train_df = pd.DataFrame([ID, Label, Sentence]).T

In [107]:
train_df.columns = ['ID', 'Label', 'Sentence']

In [117]:
train_df[:30]

,ID,Label,Sentence
0,train_000000,0,dung được sản_phẩm tốt cám_ơn cửa_hàng đóng_gó...
1,train_000001,0,chất_lượng sản_phẩm tuyệt_vời son mịn nhưng kh...
2,train_000002,0,chất_lượng sản_phẩm tuyệt_vời nhưng không có h...
3,train_000003,1,mình hơi thất_vọng 1 chút vì mình đã kỳ_vọng c...
4,train_000004,1,lần trước mình mua áo_gió màu hồng rất o không...
5,train_000005,0,chất_lượng sản_phẩm tuyệt_vời có_điều không cứ...
6,train_000006,0,đã nhận được hàng rất nhanh mới đặt buổi tối m...
7,train_000007,1,các siêu phẩm thấy cấu_hình toàn tựa tựa nhau ...
8,train_000008,0,hàng giao hàng nhanh chất_lượng tốt tư_vấn nhi...
9,train_000009,1,đồng_hồ đẹp nhưng 1 cái đứt dây 1 cái không ch...


In [109]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16091 entries, 0 to 16090
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   ID        16091 non-null  object
 1   Label     16091 non-null  object
 2   Sentence  16091 non-null  object
dtypes: object(3)
memory usage: 377.3+ KB


### Split train/test data

In [110]:
from sklearn.model_selection import train_test_split

In [111]:
x_train, x_test, y_train, y_test = train_test_split(train_df.loc[:,'Sentence'].values, (train_df.loc[:,'Label'].values).astype('int'), test_size = .2, random_state=0)

### Multinomial Naive Bayes

In [112]:
model = NaiveBayesModel()
clf = model.clf.fit(x_train, y_train)
print("Train accuracy:",clf.score(x_train, y_train))
print("Test accuracy:", clf.score(x_test, y_test))

Train accuracy: 0.8980733374766936
Test accuracy: 0.8657968313140727


### SVM kernel = 'linear'

In [113]:
clf_svm = SVMModel().clf.fit(x_train, y_train)
print("Train accuracy:",clf_svm.score(x_train, y_train))
print("Test accuracy:", clf_svm.score(x_test, y_test))

Train accuracy: 0.9377719080174021
Test accuracy: 0.8800869835352594


## Input from keyboard

In [150]:
sentiment = ['tích cực', 'tiêu cực']
test_data = {}
test_data['Sentence']= input('Mời bạn nhập bình luận: ')
test_df = pd.DataFrame(test_data,index=[0])
predict = clf_svm.predict(test_df['Sentence'])[0]
print(sentiment[predict])

Mời bạn nhập bình luận: amazing goot chop em
tích cực


### Filter error label predict

In [122]:
predict_label = clf_svm.predict(train_df['Sentence'])

In [124]:
error_predict_df = train_df[train_df['Label']!= predict_label]

In [139]:
length = []
for i in range(error_predict_df.shape[0]):
    length.append(len((error_predict_df['Sentence'][error_predict_df.index[i]]).split()))
error_predict_df['Length_sentence'] = length

c:\users\manhlk\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [140]:
error_predict_df[-10:]

,ID,Label,Sentence,Length_sentence
15958,train_015958,0,vì không đúng mẫu và màu_da nên không_thể đánh...,10
15961,train_015961,0,sản_phẩm rất o không,4
15963,train_015963,1,quạt đẹp giá tốt rất hài_lòng với sản pham,9
15976,train_015976,0,hồi đó mình xài s5 nhưng chạy theo phong_trào ...,92
15993,train_015993,0,như l,2
16029,train_016029,0,rất ấn_tượng,2
16087,16088,0,5star,1
16088,16089,0,positive,1
16089,16090,1,1star,1
16090,16091,1,negative,1


### Export to Excel

In [144]:
false_predict_data = pd.ExcelWriter('error_predict.xlsx',engine='xlsxwriter')
error_predict_df.to_excel(false_predict_data,sheet_name='Sheet1')
false_predict_data.save()

In [146]:
error_predict_df['Length_sentence'].describe()

count    1191.000000
mean       15.477750
std        14.857111
min         0.000000
25%         6.000000
50%        11.000000
75%        20.000000
max       153.000000
Name: Length_sentence, dtype: float64